In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 21193549
paper_name = 'decastro_goldman_2011' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/Supplementary_Table_2S.xlsx', sheet_name='Plan2')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 159 x 3


In [7]:
original_data.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2
0,Metabolic process (GO:0008152),NaN,NaN
1,THR1,YHR025W,"Homoserine kinase, conserved protein required ..."
2,HOM6,YJR139C,Homoserine dehydrogenase (L-homoserine:NADP ox...
3,NONE,YGR012W,"Putative cysteine synthase, localized to the m..."
4,TRP3,YKL211C,Bifunctional enzyme exhibiting both indole-3-g...


In [8]:
original_data['orf'] = original_data['Unnamed: 1'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

                                                    Unnamed: 0 Unnamed: 1  \
index_input                                                                 
0                               Metabolic process (GO:0008152)        NaN   
28                                                         NaN        NaN   
29           Protein Transport (GO:0015031) and Transport (...        NaN   
67                                                         NaN        NaN   
68           Cellular component organization and biogenesis...        NaN   
80                                                         NaN        NaN   
81           DNA metabolic process (GO:0006259) and Transcr...        NaN   
110                                                        NaN        NaN   
111                                RNA processing (GO:0006396)        NaN   
120                                                        NaN        NaN   
121                                  Translation (GO:0006412;)        NaN   

In [12]:
original_data = original_data.loc[t,:]

In [13]:
original_data['data'] = -1

In [14]:
original_data.set_index('orf', inplace=True)

In [15]:
original_data = original_data[['data']].copy()

In [16]:
original_data = original_data.groupby(original_data.index).mean()

In [17]:
original_data.shape

(138, 1)

# Load & process tested strains

In [18]:
tested = pd.read_csv('raw_data/strain_a_mating_type.txt', sep='\t')

In [19]:
tested.head()

,rec_num,ORF_name,deletion_alias,A_confirmation_primer_sequence,B_confirmation_primer_sequence,C_confirmation_primer_sequence,D_confirmation_primer_sequence,UPTAG_primer_sequence,DNTAG_primer_sequence,UPstream45_primer_sequence,...,AD_wt,AD_del,AB_del_PCR,CD_del_PCR,UPTAG_sequence_20mer,DNTAG_sequence_20mer,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
0,=====,=========,==============,==============================,==============================,==============================,==============================,==============================================...,==============================================...,==============================================...,...,========,========,==========,==========,====================,====================,NaN,NaN,NaN,NaN
1,405,YAL002W,,CTTCATATACGGCACTCCCCTGAA,CTCAGATAAGGTCGGCACTAGGATTG,ATGTACAGCGCCACCAAGATATGA,TAGGTGTCGTGAAGTACTTTCGCTT,GATGTCCACGAGGTCTCTATACTGACAGCACGCATGGCCGTACGCT...,...,GGAGGGCCCACGTGCTTGTATCCGACAGCCACGTATTTTTTGATGG...,...,3904,1918,,,ATACTGACAGCACGCATGGC,NaN,NaN,NaN,NaN,NaN
2,402,YAL004W,,AATCTGGCTCTGGTGATGGAAGTGTA,TCTCTTTGTTGTTCATTGAAGACGGTA,AGATTCGGCAGTTTCCTTCATCTTACC,TTAGCTGATCGTTTCGAGGACTTCAA,GATGTCCACGAGGTCTCTTATGGCACGGCAGACATTCCCGTACGCT...,...,GAAGTGGTTGACCAATCTGTTGTCAAAATCTTCACCACCCAAATGG...,...,1094,1991,,,TATGGCACGGCAGACATTCC,NaN,NaN,NaN,NaN,NaN
3,403,YAL005C,,GTGGATTGATGTATCTTCGAGAAGG,AAGCGACAAAAGATGGAGTGGTTC,TAAGTTGTACCAAGCTGGTGGTGCTC,AGCAGTACTTCAACCATTAGTGTTAGCG,GATGTCCACGAGGTCTCTAGGCATACTACACAGATTCCCGTACGCT...,...,AAGTATTACAAGAAACAAAAATTCAAGTAAATAACAGATAATATGG...,...,2300,1916,,,AGGCATACTACACAGATTCC,NaN,NaN,NaN,NaN,NaN
4,401,YAL007C,,TCATTAACCTTAGGGATCGACACC,CTATTTCCTCCACGGCGTTATTGG,GCCGTTATAAGTATTGCCCAAGTTCT,AACAAATGAGCAAGACAAATGACCAG,GATGTCCACGAGGTCTCTGAGTGATCCATACACATTCCCGTACGCT...,...,AATAAGATAAACAGTGGAGTACACTCATTGTAAAGGAAAACCATGG...,...,943,1840,,,GAGTGATCCATACACATTCC,NaN,NaN,NaN,NaN,NaN


In [20]:
tested['orf'] = tested['ORF_name'].astype(str)

In [21]:
tested['orf'] = clean_orf(tested['orf'])

In [ ]:
tested.loc[tested['orf']=='YLR287-A','orf'] = 'YLR287C-A'

In [22]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [23]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

            rec_num   ORF_name  deletion_alias  \
index_input                                      
0             =====  =========  ==============   

             A_confirmation_primer_sequence  B_confirmation_primer_sequence  \
index_input                                                                   
0            ==============================  ==============================   

             C_confirmation_primer_sequence  D_confirmation_primer_sequence  \
index_input                                                                   
0            ==============================  ==============================   

            UPTAG_primer_sequence                                                       \
index_input                                                                              
0            ==============================================...                           

            DNTAG_primer_sequence                                                        \
index_input

In [24]:
tested = tested.loc[t,:]

In [25]:
tested_orfs = tested['orf'].unique()

In [26]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

[]

In [27]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [28]:
data = original_data.copy()

In [29]:
dataset_ids = [150]
datasets = datasets.reindex(index=dataset_ids)

In [30]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [31]:
data.head()

dataset_id,150
data_type,value
orf,
YAL002W,0
YAL004W,0
YAL005C,0
YAL007C,0
YAL008W,0


## Subset to the genes currently in SGD

In [32]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 23


In [33]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,150
,data_type,value
gene_id,orf,
2,YAL002W,0
1863,YAL004W,0
4,YAL005C,0
5,YAL007C,0
6,YAL008W,0


# Normalize

In [34]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [35]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [36]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        150       
data_type       value valuez
gene_id orf                 
2       YAL002W     0    0.0
1863    YAL004W     0    0.0
4       YAL005C     0    0.0
5       YAL007C     0    0.0
6       YAL008W     0    0.0

# Print out

In [37]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [38]:
from IO.save_data_to_db3 import *

In [39]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 21193549...
Inserting the new data...


100%|██████████| 1/1 [00:08<00:00,  8.03s/it]

Updating the data_modified_on field...
